# Instructor Do: Decision Trees  -Random Forest

In [1]:
# Initial imports
import pandas as pd
from path import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

## Loading and Preprocessing Loans Encoded Data

In [2]:
# Loading data
file_path = Path("../Resources/loans_data_encoded.csv")
df_loans = pd.read_csv(file_path)
df_loans.head()

,amount,term,age,bad,month_num,education_Bachelor,education_High School or Below,education_Master or Above,education_college,gender_female,gender_male
0,1000,30,45,0,6,0,1,0,0,0,1
1,1000,30,50,0,7,1,0,0,0,1,0
2,1000,30,33,0,8,1,0,0,0,1,0
3,1000,15,27,0,9,0,0,0,1,0,1
4,1000,30,28,0,10,0,0,0,1,1,0


In [3]:
# Define features set
X = df_loans.copy()
X = X.drop("bad", axis=1)
X.head()

,amount,term,age,month_num,education_Bachelor,education_High School or Below,education_Master or Above,education_college,gender_female,gender_male
0,1000,30,45,6,0,1,0,0,0,1
1,1000,30,50,7,1,0,0,0,1,0
2,1000,30,33,8,1,0,0,0,1,0
3,1000,15,27,9,0,0,0,1,0,1
4,1000,30,28,10,0,0,0,1,1,0


In [9]:
# Define target vector
y = df_loans["bad"].ravel()
y[:5]

array([0, 0, 0, 0, 0])

In [49]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [50]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(375, 6)
(125, 6)
(375,)
(125,)


In [51]:
# Creating a StandardScaler instance
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data
X_scaler = scaler.fit(X_train)
# Scaling the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Fitting the Random Forest Model

In [52]:
# Create a random forest classifier
rf_model = RandomForestClassifier(n_estimators = 500, random_state = 78)

In [53]:
#Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

## Making Predictions Using the Tree Model

In [54]:
predicitions = rf_model.predict(X_test_scaled)
predicitions

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0])

## Model Evaluation

In [55]:
#Calculate the confusion matrix
cm = confusion_matrix(predicitions, y_test)
cm_df = pd.DataFrame(cm, index=['Actual 0', 'Actual 1'], columns=['Predicted 0', 'Predicted 1'])
cm_df

,Predicted 0,Predicted 1
Actual 0,54,28
Actual 1,30,13


In [56]:
#Set accuracy test
acc_score = accuracy_score(predicitions, y_test)
print(f"Accuracy is {acc_score}")

Accuracy is 0.536


In [57]:
#Classification report
cp = classification_report(y_test, predicitions)
print(f"Classification report \n{cp}")

Classification report 
              precision    recall  f1-score   support

           0       0.66      0.64      0.65        84
           1       0.30      0.32      0.31        41

    accuracy                           0.54       125
   macro avg       0.48      0.48      0.48       125
weighted avg       0.54      0.54      0.54       125



# Ranking Features

In [58]:
# Calculate feature inportance in the Random Forest model
importances = rf_model.feature_importances_
importances


array([0.05527956, 0.083377  , 0.47627742, 0.31881675, 0.03075444,
       0.03549482])

In [59]:
# We can sort the features by their importance
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.4762774155433003, 'age'),
 (0.3188167541048824, 'month_num'),
 (0.08337700488414826, 'term'),
 (0.05527956048657813, 'amount'),
 (0.03549482315572748, 'education_college'),
 (0.030754441825363306, 'education_High School or Below')]

In [47]:
#drop lower rank features (gender, education bachelor, education master above)
X = X.drop(["gender_male", "gender_female", "education_Bachelor", "education_Master or Above"], axis=1)
X

,amount,term,age,month_num,education_High School or Below,education_college
0,1000,30,45,6,1,0
1,1000,30,50,7,0,0
2,1000,30,33,8,0,0
3,1000,15,27,9,0,1
4,1000,30,28,10,0,1
...,...,...,...,...,...,...
495,1000,30,28,12,1,0
496,1000,15,26,7,1,0
497,800,15,30,6,0,1
498,1000,30,38,3,0,1
